In [1]:
# clone repo
!git clone -b development --single-branch https://github.com/andreazenotto/AML_Project_Anomaly_Segmentation.git

Cloning into 'AML_Project_Anomaly_Segmentation'...
remote: Enumerating objects: 14849, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 14849 (delta 21), reused 25 (delta 14), pack-reused 14809 (from 3)
Receiving objects: 100% (14849/14849), 754.94 MiB | 48.71 MiB/s, done.
Resolving deltas: 100% (3689/3689), done.
Updating files: 100% (507/507), done.


In [2]:
# download cityscapes
!kaggle datasets download -d andreazenotto/cityscapes
!unzip -q "cityscapes.zip" -d "AML_Project_Anomaly_Segmentation/datasets"

Dataset URL: https://www.kaggle.com/datasets/andreazenotto/cityscapes-train
License(s): MIT
100%|█████████████████████████████████████▉| 6.71G/6.71G [02:03<00:00, 85.9MB/s]
100%|██████████████████████████████████████| 6.71G/6.71G [02:03<00:00, 58.4MB/s]
Dataset URL: https://www.kaggle.com/datasets/andreazenotto/cityscapes-test-val
License(s): MIT
100%|█████████████████████████████████████▉| 4.37G/4.38G [00:48<00:00, 92.2MB/s]
100%|██████████████████████████████████████| 4.38G/4.38G [00:48<00:00, 97.1MB/s]


In [3]:
%%capture
!pip3 install visdom
!pip3 install ood_metrics
!pip3 install torchvision

In [4]:
models = {'erfnet': ['erfnet_pretrained.pth', 'erfnet.py', 'erfnet'],
          'enet': ['enet_pretrained.pth', 'enet.py', 'enet'],
          'bisenet': ['bisenet_pretrained.pth', 'bisenet.py', 'bisenet'],
          'erfnet_ce' : ['erfnet_ce.pth', 'erfnet.py', 'erfnet'],
          'erfnet_eim': ['erfnet_eim.pth', 'erfnet.py', 'erfnet'],
          'erfnet_ln': ['erfnet_ln.pth', 'erfnet.py', 'erfnet'],
          'erfnet_mixeim': ['erfnet_mixeim.pth', 'erfnet.py', 'erfnet'],
          'erfnet_mixln': ['erfnet_mixln.pth', 'erfnet.py', 'erfnet'],
          'erfnet_mixall': ['erfnet_mixall.pth', 'erfnet.py', 'erfnet'],}

def inference_eval(model, method, dataset, temp=1):
    if dataset == 'cityscapes':
        !python3 AML_Project_Anomaly_Segmentation/eval/eval_iou.py --loadWeights {model[0]} --loadModel {model[1]} --method {method} --temp {temp}
    elif dataset == 'ra-21':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/RoadAnomaly21/images/*.png' --loadWeights {model[0]} --loadModel {model[1]} --method {method} --temp {temp}
    elif dataset == 'ro-21':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/RoadObsticle21/images/*.webp' --loadWeights {model[0]} --loadModel {model[1]} --method {method} --temp {temp}
    elif dataset == 'fs-lf':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/FS_L&F/images/*.png' --loadWeights {model[0]} --loadModel {model[1]} --method {method} --temp {temp}
    elif dataset == 'fs-static':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/FS_Static/images/*.jpg' --loadWeights {model[0]} --loadModel {model[1]} --method {method} --temp {temp}
    elif dataset == 'ra':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/RoadAnomaly/images/*.jpg' --loadWeights {model[0]} --loadModel {model[1]} --method {method} --temp {temp}


def void_classifier(model, dataset):
    if dataset == 'cityscapes':
        !python3 AML_Project_Anomaly_Segmentation/eval/eval_iou.py --loadWeights {model[0]} --loadModel {model[1]} --void
    elif dataset == 'ra-21':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/RoadAnomaly21/images/*.png' --loadWeights {model[0]} --loadModel {model[1]}  --void
    elif dataset == 'ro-21':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/RoadObsticle21/images/*.webp' --loadWeights {model[0]} --loadModel {model[1]}  --void
    elif dataset == 'fs-lf':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/FS_L&F/images/*.png' --loadWeights {model[0]} --loadModel {model[1]}  --void
    elif dataset == 'fs-static':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/FS_Static/images/*.jpg' --loadWeights {model[0]} --loadModel {model[1]}  --void
    elif dataset == 'ra':
        !python3 AML_Project_Anomaly_Segmentation/eval/evalAnomaly.py --input 'AML_Project_Anomaly_Segmentation/datasets/RoadAnomaly/images/*.jpg' --loadWeights {model[0]} --loadModel {model[1]}  --void

def train_model(model, epochs):
    !python3 AML_Project_Anomaly_Segmentation/train/main.py --model {model[2]} --state AML_Project_Anomaly_Segmentation/trained_models/{model[2]}_training_weights.pth --savedir AML_Project_Anomaly_Segmentation/training_results --num-epochs {epochs} --decoder

def train_model_from_checkpoint(model, epochs):
    !python3 AML_Project_Anomaly_Segmentation/train/main.py --model {model[2]} --state AML_Project_Anomaly_Segmentation/training_results/model_best.pth --savedir AML_Project_Anomaly_Segmentation/training_results --num-epochs {epochs} --decoder  --resume

def train_erfnet_with_loss(epochs, loss):
    !python3 AML_Project_Anomaly_Segmentation/train/main.py --num-epochs {epochs} --savedir AML_Project_Anomaly_Segmentation/training_results --decoder --pretrainedEncoder AML_Project_Anomaly_Segmentation/trained_models/erfnet_encoder_pretrained.pth.tar --loss {loss}


# Erfnet Inference

In [5]:
datasets = ['cityscapes', 'ra-21', 'ro-21', 'fs-lf', 'fs-static', 'ra']
methods = ['msp', 'ml', 'me']

for method in methods:
    print(f'\nMethod: {method}')
    for dataset in datasets:
        print(f'Dataset: {dataset}')
        inference_eval(models['erfnet'], method, dataset)


Method: msp
Dataset: cityscapes
---------------------------------------
Took  54.82676959037781 seconds
Per-Class IoU:
97.62 Road
81.37 sidewalk
90.77 building
49.43 wall
54.93 fence
60.81 pole
62.60 traffic light
72.31 traffic sign
91.35 vegetation
60.96 terrain
93.38 sky
76.11 person
53.45 rider
92.91 car
72.78 truck
78.87 bus
63.86 train
46.40 motorcycle
71.89 bicycle
MEAN IoU:  72.20 %
Dataset: ra-21
AUPRC score: 29.095121622764314
FPR@TPR95: 62.548789859326625
Dataset: ro-21
AUPRC score: 2.7099746270831964
FPR@TPR95: 65.22309081830707
Dataset: fs-lf
AUPRC score: 1.7490388138046733
FPR@TPR95: 50.59403252456551
Dataset: fs-static
AUPRC score: 7.472042122551044
FPR@TPR95: 41.83689056458776
Dataset: ra
AUPRC score: 12.422651139450682
FPR@TPR95: 82.57546412366915

Method: ml
Dataset: cityscapes
---------------------------------------
Took  54.749566078186035 seconds
Per-Class IoU:
97.62 Road
81.37 sidewalk
90.77 building
49.43 wall
54.93 fence
60.81 pole
62.60 traffic light
72.31 traf

# Erfnet Inference with MSP and Temperature

In [6]:
datasets = ['ra-21', 'ro-21', 'fs-static', 'fs-lf', 'ra']
temps = [0.5, 0.75, 1.1]

for dataset in datasets:
    print(f'\nDataset: {dataset}\n')
    for temp in temps:
        print(f'\nTemperature: {temp}\n')
        inference_eval(models['erfnet'], 'msp', dataset, temp)


Dataset: ra-21


Temperature: 0.5

AUPRC score: 27.060832506682093
FPR@TPR95: 62.730810427606734

Temperature: 0.75

AUPRC score: 28.155279219746888
FPR@TPR95: 62.48680866415934

Temperature: 1.1

AUPRC score: 29.40109502990522
FPR@TPR95: 62.64680449239919

Dataset: ro-21


Temperature: 0.5

AUPRC score: 2.419550221540522
FPR@TPR95: 63.22544524787239

Temperature: 0.75

AUPRC score: 2.566576576220588
FPR@TPR95: 64.1288367459823

Temperature: 1.1

AUPRC score: 2.7630918639701547
FPR@TPR95: 65.86731905563137

Dataset: fs-static


Temperature: 0.5

AUPRC score: 6.601199824046117
FPR@TPR95: 43.47565874225287

Temperature: 0.75

AUPRC score: 6.991020312229113
FPR@TPR95: 42.49932670115177

Temperature: 1.1

AUPRC score: 7.690238094059254
FPR@TPR95: 41.62191102506234

Dataset: fs-lf


Temperature: 0.5

AUPRC score: 1.2802520336004974
FPR@TPR95: 66.73710676943257

Temperature: 0.75

AUPRC score: 1.492986422527121
FPR@TPR95: 51.76136514478542

Temperature: 1.1

AUPRC score: 1.8614808647279373


## Finding best temperature
Initially starting from t = 1.1 to t = 5, to understand their nature with different temperature values:
- From t = 1.1 to t = 2 with a step of 0.1
- From t = 2 to t = 5 with a step of 0.5

In [6]:

datasets = ['cityscapes', 'ra-21', 'ro-21', 'fs-static', 'fs-lf', 'ra']
temp = 1

while temp < 5:
    if temp < 2:
        temp += 0.1
    else:
        temp += 0.5
    print(f'\nTemperature: {temp}')
    for dataset in datasets:
        print(f'\nDataset: {dataset}')
        inference_eval(models['erfnet'], 'msp', dataset, temp)



Temperature: 1.1

Dataset: cityscapes
---------------------------------------
Took  54.00762987136841 seconds
Per-Class IoU:
97.62 Road
81.37 sidewalk
90.77 building
49.43 wall
54.93 fence
60.81 pole
62.60 traffic light
72.31 traffic sign
91.35 vegetation
60.96 terrain
93.38 sky
76.11 person
53.45 rider
92.91 car
72.78 truck
78.87 bus
63.86 train
46.40 motorcycle
71.89 bicycle
MEAN IoU:  72.20 %

Dataset: ra-21
AUPRC score: 29.40109502990522
FPR@TPR95: 62.64680449239919

Dataset: ro-21
AUPRC score: 2.7630918639701547
FPR@TPR95: 65.86731905563137

Dataset: fs-static
AUPRC score: 7.690238094059254
FPR@TPR95: 41.62191102506234

Dataset: fs-lf
AUPRC score: 1.8614808647279373
FPR@TPR95: 50.17685456318127

Dataset: ra
AUPRC score: 12.460467813627002
FPR@TPR95: 82.73353126265913

Temperature: 1.2000000000000002

Dataset: cityscapes
---------------------------------------
Took  54.96257972717285 seconds
Per-Class IoU:
97.62 Road
81.37 sidewalk
90.77 building
49.43 wall
54.93 fence
60.81 pole


We can see we have good performances in a range between t = 1.7 and t = 1.9:
- FS-Static has a very low FPR
- Ra21, Ro21 and Ra are very close to their maximum in terms of AUPRC
- FS-L&F continue to grow outside this range, but other datasets ( Ra21, Ro21, Ra ) does not

So we can perform a test in this range with a step of 0.5 to choose between the range [1.7, 1.8] and [1.8, 1.9]

In [6]:
# we do not include here cityscapes since temperature does not improve performances of mIoU
# we forgot to remove from the above cell datasets list
datasets = ['ra-21', 'ro-21', 'fs-static', 'fs-lf', 'ra']
temp = 1.7

while temp < 1.9:
    temp += 0.05
    print(f'\nTemperature: {temp}\n')
    for dataset in datasets:
        print(f'\nDataset: {dataset}\n')
        inference_eval(models['erfnet'], 'msp', dataset, temp)


Temperature: 1.75


Dataset: ra-21

AUPRC score: 30.482385124285976
FPR@TPR95: 64.18193525550817

Dataset: ro-21

AUPRC score: 2.9802127858893033
FPR@TPR95: 70.65790029345911

Dataset: fs-static

AUPRC score: 9.106858891758074
FPR@TPR95: 41.14392230691502

Dataset: fs-lf

AUPRC score: 2.5122162684103775
FPR@TPR95: 48.487317619286415

Dataset: ra

AUPRC score: 12.619576086183276
FPR@TPR95: 84.02956894304666

Temperature: 1.8


Dataset: ra-21

AUPRC score: 30.516293426776436
FPR@TPR95: 64.34612530684662

Dataset: ro-21

AUPRC score: 2.986904461405072
FPR@TPR95: 71.11376591502273

Dataset: fs-static

AUPRC score: 9.200076471786062
FPR@TPR95: 41.1416576378217

Dataset: fs-lf

AUPRC score: 2.5507041020801533
FPR@TPR95: 48.38155049507687

Dataset: ra

AUPRC score: 12.625962046442218
FPR@TPR95: 84.13864141062517

Temperature: 1.85


Dataset: ra-21

AUPRC score: 30.545388692386048
FPR@TPR95: 64.5270033143958

Dataset: ro-21

AUPRC score: 2.9924206837703164
FPR@TPR95: 71.54061063342122

Datase

Our final choice is t = 1.7. With this temperature we can minimize the FPR and still have a good AUPRC (as we saw before)

# Training ENet and BiSeNet

In [ ]:
# train_model(models['enet'], epochs=150)

In [ ]:
# train_model_from_checkpoint(models['erfnet'], epochs=150)

In [ ]:
# train_model(models['bisenet'], epochs=150)

# Void Classifier

In [5]:
datasets = ['cityscapes', 'ra-21', 'ro-21', 'fs-lf', 'fs-static', 'ra']
model_names = ['erfnet', 'enet', 'bisenet']

for model in model_names:
    print(f'\nModel: {model}\n')
    for dataset in datasets:
        print(f'Dataset: {dataset}\n')
        void_classifier(models[model], dataset)


Model: erfnet

Dataset: cityscapes

---------------------------------------
Took  54.60863518714905 seconds
Per-Class IoU:
81.20 Road
66.53 sidewalk
83.70 building
33.24 wall
39.92 fence
54.85 pole
57.22 traffic light
62.51 traffic sign
88.20 vegetation
43.02 terrain
86.58 sky
69.79 person
52.12 rider
86.95 car
58.73 truck
61.56 bus
41.94 train
30.46 motorcycle
65.18 bicycle
0.00 void
MEAN IoU:  58.19 %
Dataset: ra-21

AUPRC score: 34.346714749908145
FPR@TPR95: 53.35673907431191
Dataset: ro-21

AUPRC score: 4.848900458350778
FPR@TPR95: 14.212955755905615
Dataset: fs-lf

AUPRC score: 0.5898383050388653
FPR@TPR95: 69.86855126915468
Dataset: fs-static

AUPRC score: 4.074303302459759
FPR@TPR95: 65.70011167021119
Dataset: ra

AUPRC score: 29.605370095409917
FPR@TPR95: 62.78188003051171

Model: enet

Dataset: cityscapes

---------------------------------------
Took  63.49655079841614 seconds
Per-Class IoU:
83.54 Road
59.52 sidewalk
78.36 building
35.46 wall
32.25 fence
43.32 pole
39.26 traf

# Effect of Training Loss function

## Enhanced Isotropy Maximization Loss

In [ ]:
# train_erfnet_with_loss(50, 'eim')

## Logit Normalization loss

In [ ]:
# train_erfnet_with_loss(50, 'ln')

## Enhanced Isotropy Maximization Loss + Cross Entopy and Focal Loss

In [ ]:
# train_erfnet_with_loss(50, 'mixeim')

## Logit Normalization Loss + Cross Entopy and Focal Loss

In [ ]:
# train_erfnet_with_loss(50, 'mixln')

## Enhanced Isotropy Maximization Loss + Logit Normalization + Cross Entopy and Focal Loss

In [ ]:
# train_erfnet_with_loss(50, 'mixall')

# Inference with Different Training Loss

In [17]:
datasets = ['cityscapes', 'ra-21', 'ro-21', 'fs-lf', 'fs-static', 'ra']
methods = ['msp', 'ml', 'me']
model_names = ['erfnet_ce', 'erfnet_eim', 'erfnet_ln', 'erfnet_mixeim', 'erfnet_mixln', 'erfnet_mixall']

for model in model_names:
    print(f'\nModel: {model}\n')
    for method in methods:
        print(f'Method: {method}\n')
        for dataset in datasets:
            print(f'Dataset: {dataset}\n')
            inference_eval(models[model], method, dataset)


Model: erfnet_ce

Method: msp

Dataset: cityscapes

---------------------------------------
Took  56.00537919998169 seconds
Per-Class IoU:
96.88 Road
77.79 sidewalk
88.94 building
40.89 wall
53.24 fence
54.95 pole
55.08 traffic light
68.10 traffic sign
90.15 vegetation
60.56 terrain
92.00 sky
70.14 person
51.77 rider
91.02 car
60.16 truck
66.28 bus
44.36 train
41.36 motorcycle
67.90 bicycle
MEAN IoU:  66.92 %
Dataset: ra-21

AUPRC score: 24.305324590541005
FPR@TPR95: 94.49875069383347
Dataset: ro-21

AUPRC score: 1.9051361176583028
FPR@TPR95: 49.209455196147275
Dataset: fs-lf

AUPRC score: 2.462070783740886
FPR@TPR95: 64.44486133492539
Dataset: fs-static

AUPRC score: 8.833402843851168
FPR@TPR95: 36.49793884312543
Dataset: ra

AUPRC score: 13.256246245697376
FPR@TPR95: 81.83751693998197
Method: ml

Dataset: cityscapes

---------------------------------------
Took  53.968374252319336 seconds
Per-Class IoU:
96.88 Road
77.79 sidewalk
88.94 building
40.89 wall
53.24 fence
54.95 pole
55.08